# Análisis Sistema de recomendacion

El enriquecimiento de las descripciones de los productos impacta positivamente en los motores de recomendación, proporcionando información específica y completa orientada al usuario final. Esto facilita que los compradores puedan tomar decisiones informadas sin necesidad de buscar información adicional por fuera de la plataforma.

Es fundamental que los sistemas obliguen el llenado de ciertos atributos claves para maximizar la precisión de las descripciones generadas por IA, optimizando así la experiencia de búsqueda y recomendación.

---

#Descripción General de la Solución Técnica

Se desarrolló  ETL en Python que realiza la extracción automática de productos desde la API de Meli, aplicando filtros por dominio y palabras clave. En este caso, se utilizó el dominio de alimentos para mascotas.

El flujo consiste en:

- Autenticación y extracción de productos desde MercadoLibre vía API.
- Filtrado y estructuración de los atributos extraídos.
- Construcción de un JSON estructurado a partir de los atributos disponibles.
- Generación de descripciones enriquecidas mediante OpenAI (modelo gpt-4o), utilizando un prompt orientado a productos de mascotas.
- Almacenamiento de los resultados en un DataFrame para visualización y posibles cargas posteriores.

Se toman únicamente los atributos entregados por la API para evitar inconsistencias, manteniendo un tono empático y persuasivo acorde al público objetivo.

---

#  Desarrollo en Python


In [ ]:
# Importo librerias necesarias
import requests
import pandas as pd
import re
import json
import openai
from google.colab import userdata

In [ ]:
!pip show openai

Name: openai
Version: 1.84.0
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [ ]:
client_id_meli = userdata.get('client_id_meli')
client_secret_meli = userdata.get('client_secret_meli')
client_openia = userdata.get('key_openia').strip()
client = openai.OpenAI(api_key=client_openia)

In [ ]:
def get_token_meli():
    url = "https://api.mercadolibre.com/oauth/token"
    payload = {
        "grant_type": "client_credentials",
        "client_id": userdata.get('client_id_meli'),
        "client_secret": userdata.get('client_secret_meli')
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    response = requests.post(url, data=payload, headers=headers)
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        print("Error al generar token:", response.text)
        return None

Buscamos poductos por dominio Alimento para mascotas y especificamos en la query que sea de tipo purina

In [ ]:
def products_by_domain(token, site_id="MLA", query="alimento purina", domain_id="MLA-CAT_AND_DOG_FOODS", limite=20):
    url = "https://api.mercadolibre.com/products/search"
    headers = {
        'Authorization': f'Bearer {token}',
        'User-Agent': 'Descripciones ML'
    }
    params = {
        'status': 'active',
        'site_id': site_id,
        'q': query,
        'domain_id': domain_id,
        'limit': limite
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json().get("results", [])
        if not data:
            print("No se encontraron productos para los parámetros indicados.")
        return data

    except requests.exceptions.RequestException as err:
        print("Error inesperado:", err)
        return []

Preparación de datos para el modelo de IA
Previo a pegarle a  la API de OpenAI, estructuramos un JSON para cada producto que contiene:
El nombre del producto (Producto)
El identificador del producto (id)
Los atributos detallados.
De esta forma, el modelo recibe únicamente la información de la descripcion del producto reduciendo la posibilidad que la IA se confunda pasandole mas campos innecesarios.

In [ ]:
def prev_json_openai(producto):
    datos = {
        "Producto": producto.get("name", ""),
        "id": producto.get("id", "")
    }
    for atributo in producto.get("attributes", []):
        nombre_atributo = atributo.get("name", "").strip()
        valor = atributo.get("value_name", "N/A")
        datos[nombre_atributo] = valor
    return datos

Una vez preparados los datos de cada producto en el JSON, se procede a generar la descripción utilizando el modelo de lenguaje de OpenAI, se estructura la salida con
-Un título llamativo
-Introducción general de beneficios
-Tres bullets con ventajas concretas del producto
-Un cierre persuasivo


In [ ]:
def generate_description(producto_json):
    prompt = f"""
You are a professional e-commerce copywriter. Your task is to create a warm, clear, and engaging product description for a pet food item, written in Spanish. The description should be directed at people who care deeply about their pets and want the best for their health and well-being. Highlight the real benefits the product offers.

Instructions:
- Do not invent information. If any attribute is missing or marked as "N/A", simply omit it.
- Maintain a warm, empathetic, and trustworthy tone, without exaggerating.
- Avoid repeating the product name unnecessarily.
- If two fields contain similar or redundant information, use only one.
- Focus on clarity, practical benefits, and emotional connection.

Output format:
1. Catchy title
2. Introductory paragraph with general benefits
3. Three bullet points with real product advantages
4. Closing line with a persuasive call to action

{json.dumps(producto_json, indent=2)}

Descripción:"""

    try:
        respuesta = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a professional e-commerce."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        return respuesta.choices[0].message.content.strip()

    except Exception as e:
        print("Error al generar descripción:", e)
        return "Descripción no generada"

Se invocan las funciones previamente desarrolladas. Primero se obtiene el token de Meli, luego se consultan los productos, se estructuran los atributos relevantes y finalmente se genera la descripción con OpenAI para cada producto. Los resultados se almacenan en un dataframe estructurado que permite visualizar y analizar las descripciones generadas.


In [ ]:
def flow_full_open_ia():
    token = get_token_meli()
    if not token:
        return None

    products = products_by_domain(token)
    resultados = []

    for product in products:
        producto_struct = prev_json_openai(product)
        descripcion = generate_description(producto_struct)
        resultados.append({
            "id": producto_struct["id"],
            "producto": producto_struct["Producto"],
            "descripcion_generada": descripcion
        })

    df = pd.DataFrame(resultados)
    pd.set_option('display.max_colwidth', None)
    return df

Funcion de limpieza de algunos caracteres especiales devueltos por la api, como salto de lineas para campos vacio, **,-- etc

In [ ]:
def clean_descriptions(texto):
    if pd.isna(texto):
        return texto

    texto = texto.replace('\\n', ' ').replace('\n', ' ')
    texto = texto.replace('###', ' ')
    texto = re.sub(r'\s*\d+\.', '', texto)
    texto = texto.replace('**', '')
    texto = texto.replace('-', '')
    texto = texto.replace('\\', '')
    texto = re.sub(r'\s+', ' ', texto)
    texto = texto.strip()

    return texto

Ahora si corremos la funcion del flujo completo y aplicamos la limpieza de descripciones,
Como mejora, se propone implementar un etl incremental de procesamiento (delta), evitando procesar nuevamente los mismos IDs de productos, lo cual optimiza tanto el uso de la API como los costos de generación de descripciones. Puede ser utiizando el campo date de la api



In [ ]:
df = flow_full_open_ia()

# 2. Limpiamos df
if df is not None:
    clean_df = df.copy()
    clean_df['descripcion_generada'] = clean_df['descripcion_generada'].apply(clean_descriptions)
    pd.set_option('display.max_colwidth', None)
    display(clean_df)

,id,producto,descripcion_generada
0,MLA23122594,Alimento Purina Friskies para gato,"Delicia Nutritiva para Tu Compañero Felino Descubre un alimento que combina sabor irresistible y nutrición de calidad para tu querido gato. Este alimento está diseñado pensando en su bienestar, asegurando que cada bocado no solo sea delicioso, sino también beneficioso para su salud. Cuida de tu felino con un producto que refleja tu amor y dedicación. Nutrición Completa: Formulado para proporcionar una dieta equilibrada, asegurando que tu gato reciba todos los nutrientes esenciales que necesita para una vida activa y saludable. Ingredientes de Calidad: Elaborado con ingredientes seleccionados que no solo satisfacen el paladar de tu gato, sino que también contribuyen a su bienestar general. Energía y Vitalidad: Diseñado para mantener a tu mascota llena de energía, apoyando su curiosidad natural y sus aventuras diarias. Proporciona a tu felino el regalo de una nutrición superior. ¡Haz de cada comida un momento especial y saludable!"
1,MLA19906717,Alimento Purina Beneful Pack X20 Und Alimento Húmedo 2000g,"Delicia Nutritiva para los Cachorros Más Queridos Ofrece a tu cachorro el mejor comienzo en la vida con esta comida húmeda elaborada cuidadosamente para satisfacer sus necesidades nutricionales. Con ingredientes seleccionados y un delicioso sabor a carne y arroz, cada bocado es una celebración de salud y felicidad para tu pequeño amigo de cuatro patas. Nutrición Equilibrada: Formulada con ingredientes como pollo, hígado de cerdo y zanahorias deshidratadas, proporciona una dieta completa y balanceada para el crecimiento saludable de tu cachorro. Textura Suave y Apetecible: La textura húmeda y el sabor irresistible de carne y arroz hacen que la hora de la comida sea un momento de alegría para tu mascota. Tamaño Ideal para Pequeños Mordiscos: Diseñada especialmente para razas mini y pequeñas, asegurando que cada sobre sea fácil de manejar y perfecto para sus pequeñas bocas. Dale a tu cachorro el amor y cuidado que merece con esta deliciosa opción de alimentación. ¡Haz tu pedido hoy y observa cómo su colita no deja de moverse de felicidad!"
2,MLA20597176,"Alimento Purina Pro Plan Canine Neurocare 7,5kg","Nutrición Avanzada para el Bienestar de Tu Compañero Canino El amor incondicional que compartes con tu perro merece una nutrición que cuide cada aspecto de su salud. Por eso, hemos creado un alimento que no solo alimenta, sino que también apoya su bienestar mental y físico. Diseñado especialmente para promover la salud neurológica, este alimento seco es una elección consciente para aquellos que desean lo mejor para sus amigos de cuatro patas. Apoyo a la Salud Cerebral: Formulado con ingredientes especialmente seleccionados como el aceite de pescado y el aceite vegetal de triglicéridos de cadena media, este alimento contribuye a mantener la función cognitiva en óptimas condiciones. Ingrediente de Alta Calidad: Contiene harina de pollo y huevo seco que ofrecen proteínas esenciales, asegurando que tu perro reciba los nutrientes necesarios para una vida activa y saludable. Adecuado para Todas las Razas y Tamaños: Ya sea que tu perro sea pequeño o grande, este alimento está diseñado para satisfacer las necesidades nutricionales de perros adultos de cualquier raza. Dale a tu perro el cuidado que merece con una dieta que respalda su salud mental y física. ¡Prueba hoy mismo y observa la diferencia en su vitalidad y energía!"
3,MLA24088773,Alimento Purina Excellent Perro Adulto Cordero 4kg,"Delicias Saludables para tu Compañero Fiel Dale a tu perro adulto el amor y la nutrición que merece con un alimento que combina sabor y bienestar. Este producto está diseñado especialmente para quienes ven a sus mascotas como parte de la familia y desean proporcionarles una dieta equilibrada y deliciosa. Con una fórmula basada en cordero y arroz, cada bocado es una experiencia culinaria que favorece la salud integral de tu amigo de cuatro patas. Nutrición Adaptada para Tod